In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential

from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [ ]:
import zipfile
import os

def extract_zip(zip_path, extract_path):
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print("Extraction successful.")
    except zipfile.BadZipFile:
        print("Error: File is not a valid zip file.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
zip_path = '/content/desktop (2).zip'
extract_path = '/content'

extract_zip(zip_path, extract_path)


Extraction successful.


In [ ]:
import pandas as pd

df = pd.read_csv('/content/measurements1.csv')


In [ ]:
import os
import pandas as pd

# Load Excel data
data = pd.read_csv('/content/measurements1.csv')

# Specify the directory containing the images
image_directory = '/content/desktop/testB/mask_left'

# Create a mapping between image IDs and file paths
id_to_filepath = {}

# Iterate over the rows in the Excel sheet
for index, row in data.iterrows():
    image_id = row['Photo ID']

    # Assuming image files are named with the format "image_id.jpg"
    image_filename = f'{image_id}.png'

    # Construct the full file path
    image_filepath = os.path.join(image_directory, image_filename)

    # Check if the file exists before adding to the mapping
    if os.path.exists(image_filepath):
        id_to_filepath[image_id] = image_filepath
    else:
        print(f"Image not found for ID {image_id}")

# Now, id_to_filepath contains the mapping between image IDs and file paths
print(id_to_filepath)


{'9d409c00648ecf8303ee757698b26be9': '/content/desktop/testB/mask_left/9d409c00648ecf8303ee757698b26be9.png', 'd0b9377b4d1ece595d444d1050086507': '/content/desktop/testB/mask_left/d0b9377b4d1ece595d444d1050086507.png', 'd151534ec2286935a58485f594f95464': '/content/desktop/testB/mask_left/d151534ec2286935a58485f594f95464.png', '588af03945635d2c5cedd75e9c100a8b': '/content/desktop/testB/mask_left/588af03945635d2c5cedd75e9c100a8b.png', '38626dc8402b497be1796c5668f5f4eb': '/content/desktop/testB/mask_left/38626dc8402b497be1796c5668f5f4eb.png', '3b44bcff450d8687a983c6ef6f979756': '/content/desktop/testB/mask_left/3b44bcff450d8687a983c6ef6f979756.png', '640a12dfb3c6b7ac7cbdcc3afcd8c709': '/content/desktop/testB/mask_left/640a12dfb3c6b7ac7cbdcc3afcd8c709.png', '87d4e832af70b7750647d262ed84a0c3': '/content/desktop/testB/mask_left/87d4e832af70b7750647d262ed84a0c3.png', '42a72eab832136998f09898cb819a3f7': '/content/desktop/testB/mask_left/42a72eab832136998f09898cb819a3f7.png', '960d8574198b71acc

In [ ]:
l=[]
for i in id_to_filepath.values():
  l.append(i)
print(l)

['/content/desktop/testB/mask_left/9d409c00648ecf8303ee757698b26be9.png', '/content/desktop/testB/mask_left/d0b9377b4d1ece595d444d1050086507.png', '/content/desktop/testB/mask_left/d151534ec2286935a58485f594f95464.png', '/content/desktop/testB/mask_left/588af03945635d2c5cedd75e9c100a8b.png', '/content/desktop/testB/mask_left/38626dc8402b497be1796c5668f5f4eb.png', '/content/desktop/testB/mask_left/3b44bcff450d8687a983c6ef6f979756.png', '/content/desktop/testB/mask_left/640a12dfb3c6b7ac7cbdcc3afcd8c709.png', '/content/desktop/testB/mask_left/87d4e832af70b7750647d262ed84a0c3.png', '/content/desktop/testB/mask_left/42a72eab832136998f09898cb819a3f7.png', '/content/desktop/testB/mask_left/960d8574198b71acce45ab42d591055b.png', '/content/desktop/testB/mask_left/d55ceca8d3b04d720983bb9d2692bcb1.png', '/content/desktop/testB/mask_left/25239b1e2e3317f1d0a5e9d4be54d0df.png', '/content/desktop/testB/mask_left/f30ae7030a32f106ef0e74c9aafaf56a.png', '/content/desktop/testB/mask_left/aef90a7a904c51e0

In [ ]:
df['path2']=l
df.head()
df1=df

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, concatenate, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Assuming we have a DataFrame with columns 'path1', 'path2', 'height', 'waist', 'chest'
# Replacing those names with the actual column names in our DataFrame
data = {'path1': df['path1'],
        'path2': df['path2'],
        'height': df['height']}

df = pd.DataFrame(data)

# Function to load and preprocess images
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size, color_mode='rgb')
    img_array = image.img_to_array(img)
    img_array /= 255.0  # Rescale pixel values
    return img_array

# Apply the function to load and preprocess images
df['image1'] = df['path1'].apply(lambda x: load_and_preprocess_image(x))
df['image2'] = df['path2'].apply(lambda x: load_and_preprocess_image(x))

# Create image arrays
X1 = np.stack(df['image1'].to_numpy())
X2 = np.stack(df['image2'].to_numpy())
y_height = df['height'].to_numpy()

# Split the dataset into training and validation sets
X1_train, X1_val, X2_train, X2_val, y_height_train, y_height_val = train_test_split(
    X1, X2, y_height, test_size=0.2, random_state=42
)

# Define the VGG16 model with two inputs
input_shape_vgg16 = (224, 224, 3)
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape_vgg16)

# Freeze the convolutional layers
for layer in vgg16_model.layers:
    layer.trainable = False

# Define the ResNet50 model with two inputs
input_shape_resnet50 = (224, 224, 3)
resnet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape_resnet50)

# Freeze the convolutional layers
for layer in resnet50_model.layers:
    layer.trainable = False

# Define the DenseNet121 model with two inputs
input_shape_densenet = (224, 224, 3)
densenet_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape_densenet)

# Freeze the convolutional layers
for layer in densenet_model.layers:
    layer.trainable = False

# Define the EfficientNetB0 model with two inputs
input_shape_efficientnet = (224, 224, 3)
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape_efficientnet)

# Freeze the convolutional layers
for layer in efficientnet_model.layers:
    layer.trainable = False

# Combine the four models
input_vgg16 = Input(shape=input_shape_vgg16, name='input_vgg16')
input_resnet50 = Input(shape=input_shape_resnet50, name='input_resnet50')
input_densenet = Input(shape=input_shape_densenet, name='input_densenet')
input_efficientnet = Input(shape=input_shape_efficientnet, name='input_efficientnet')

vgg16_features = vgg16_model(input_vgg16)
resnet50_features = resnet50_model(input_resnet50)
densenet_features = densenet_model(input_densenet)
efficientnet_features = efficientnet_model(input_efficientnet)

# Concatenate the output features
merged = concatenate([Flatten()(vgg16_features), Flatten()(resnet50_features),
                      Flatten()(densenet_features), Flatten()(efficientnet_features)])

# Dense layers for further processing
x = Dense(128, activation='relu')(merged)
x = Dense(64, activation='relu')(x)

# Separate output layer for height
output_height = Dense(1, name='output_height')(x)

# Create the combined model with a single output
combined_model = Model(
    inputs=[input_vgg16, input_resnet50, input_densenet, input_efficientnet],
    outputs=[output_height]
)

# Compile the model
optimizer = Adam(learning_rate=0.001)
combined_model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])

# Train the model
history = combined_model.fit(
    {'input_vgg16': X1_train, 'input_resnet50': X1_train, 'input_densenet': X1_train, 'input_efficientnet':    X1_train},
    {'output_height': y_height_train},
    epochs=30,
    batch_size=32,
    validation_data=(
        {
            'input_vgg16': X1_val,
            'input_resnet50': X1_val,
            'input_densenet': X1_val,
            'input_efficientnet': X1_val
        },
        {'output_height': y_height_val}
    )
)



16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/30
13/13 [==============================] - 51s 2s/step - loss: 2847.4417 - mean_absolute_error: 27.9855 - val_loss: 115.0818 - val_mean_absolute_error: 8.4181
Epoch 2/30
13/13 [==============================] - 6s 466ms/step - loss: 86.4142 - mean_absolute_error: 7.4766 - val_loss: 104.2289 - val_mean_absolute_error: 8.0164
Epoch 3/30
13/13 [==============================] - 6s 474ms/step - loss: 79.7092 - mean_absolute_error: 7.1400 - val_loss: 132.1088 - val_mean_absolute_error: 9.5287
Epoch 4/30
13/13 [==============================] - 6s 476ms/step - loss: 93.9370 - mean_absolute_error: 7.8373 - val_loss: 71.5941 - val_mean_absolute_error: 6.5204
Epoch 5/30
13/13 [==============================] - 6s 480ms/step - loss: 49.2106 - mean_absolute_error: 5.5155 - val_loss: 101.2529 - val_mean_absolute_error: 8.0933
Epoch 6/30
13/13 [==============================] - 6s 465ms/step - loss: 56.7088 - mean_absolute_er

In [ ]:

image_path = "/content/desktop/testB/mask/6eedf087dddd033a288bf64a6e19d1b9.png"

input_image = load_and_preprocess_image(image_path)

input_image = np.expand_dims(input_image, axis=0)

predicted_height = combined_model.predict({
    'input_vgg16': input_image,
    'input_resnet50': input_image,
    'input_densenet': input_image,
    'input_efficientnet': input_image
})

print("Predicted Height:", predicted_height[0][0])


1/1 [==============================] - 9s 9s/step
Predicted Height: 178.64821


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, concatenate, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
df=df1

data = {'path1': df['path1'],
        'path2': df['path2'],
        'height': df['height'],
        'waist': df['waist'],
        'chest': df['chest']}

df = pd.DataFrame(data)

# Function to load and preprocess images
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size, color_mode='rgb')
    img_array = image.img_to_array(img)
    img_array /= 255.0  # Rescaling pixel values
    return img_array

# Apply the function to load and preprocess images
df['image1'] = df['path1'].apply(lambda x: load_and_preprocess_image(x))
df['image2'] = df['path2'].apply(lambda x: load_and_preprocess_image(x))

# Create image arrays
X1 = np.stack(df['image1'].to_numpy())
X2 = np.stack(df['image2'].to_numpy())
y_chest = df['chest'].to_numpy()

# Split the dataset into training and validation sets
X1_train, X1_val, X2_train, X2_val, y_chest_train, y_chest_val = train_test_split(
    X1, X2, y_chest, test_size=0.2, random_state=42
)

# Define the VGG16 model with two inputs
input_shape_vgg16 = (224, 224, 3)
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape_vgg16)

# Freeze the convolutional layers
for layer in vgg16_model.layers:
    layer.trainable = False

# Define the ResNet50 model with two inputs
input_shape_resnet50 = (224, 224, 3)
resnet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape_resnet50)

# Freeze the convolutional layers
for layer in resnet50_model.layers:
    layer.trainable = False

# Define the DenseNet121 model with two inputs
input_shape_densenet = (224, 224, 3)
densenet_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape_densenet)

# Freeze the convolutional layers
for layer in densenet_model.layers:
    layer.trainable = False

# Define the EfficientNetB0 model with two inputs
input_shape_efficientnet = (224, 224, 3)
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape_efficientnet)

# Freeze the convolutional layers
for layer in efficientnet_model.layers:
    layer.trainable = False

# Combine the four models
input_vgg16 = Input(shape=input_shape_vgg16, name='input_vgg16')
input_resnet50 = Input(shape=input_shape_resnet50, name='input_resnet50')
input_densenet = Input(shape=input_shape_densenet, name='input_densenet')
input_efficientnet = Input(shape=input_shape_efficientnet, name='input_efficientnet')

vgg16_features = vgg16_model(input_vgg16)
resnet50_features = resnet50_model(input_resnet50)
densenet_features = densenet_model(input_densenet)
efficientnet_features = efficientnet_model(input_efficientnet)

# Concatenate the output features
merged = concatenate([Flatten()(vgg16_features), Flatten()(resnet50_features),
                      Flatten()(densenet_features), Flatten()(efficientnet_features)])

# Dense layers for further processing
x = Dense(128, activation='relu')(merged)
x = Dense(64, activation='relu')(x)

# Separate output layer for chest size
output_chest = Dense(1, name='output_chest')(x)

# Create the combined model with a single output
combined_chest_model = Model(
    inputs=[input_vgg16, input_resnet50, input_densenet, input_efficientnet],
    outputs=[output_chest]
)

# Compile the model
optimizer = Adam(learning_rate=0.001)
combined_chest_model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])

# Train the model
history_chest = combined_chest_model.fit(
    {'input_vgg16': X1_train, 'input_resnet50': X1_train, 'input_densenet': X1_train, 'input_efficientnet': X1_train},
    {'output_chest': y_chest_train},
    epochs=30,
    batch_size=32,
    validation_data=(
        {
            'input_vgg16': X1_val,
            'input_resnet50': X1_val,
            'input_densenet': X1_val,
            'input_efficientnet': X1_val
        },
        {'output_chest': y_chest_val}
    )
)

Epoch 1/30
13/13 [==============================] - 31s 1s/step - loss: 1126.9789 - mean_absolute_error: 21.2679 - val_loss: 104.3855 - val_mean_absolute_error: 7.5764
Epoch 2/30
13/13 [==============================] - 6s 505ms/step - loss: 59.1252 - mean_absolute_error: 5.9608 - val_loss: 60.0606 - val_mean_absolute_error: 5.8546
Epoch 3/30
13/13 [==============================] - 6s 494ms/step - loss: 70.8373 - mean_absolute_error: 6.9284 - val_loss: 66.2705 - val_mean_absolute_error: 6.5561
Epoch 4/30
13/13 [==============================] - 6s 501ms/step - loss: 87.3235 - mean_absolute_error: 7.8325 - val_loss: 51.2895 - val_mean_absolute_error: 5.7059
Epoch 5/30
13/13 [==============================] - 6s 492ms/step - loss: 43.9961 - mean_absolute_error: 5.1853 - val_loss: 43.4340 - val_mean_absolute_error: 5.2234
Epoch 6/30
13/13 [==============================] - 6s 495ms/step - loss: 28.1187 - mean_absolute_error: 4.1117 - val_loss: 57.4280 - val_mean_absolute_error: 5.8931
Ep

In [ ]:
image_path_chest = "/content/desktop/testB/mask/6eedf087dddd033a288bf64a6e19d1b9.png"

input_image_chest = load_and_preprocess_image(image_path_chest)

input_image_chest = np.expand_dims(input_image_chest, axis=0)

predicted_chest = combined_chest_model.predict({
    'input_vgg16': input_image_chest,
    'input_resnet50': input_image_chest,
    'input_densenet': input_image_chest,
    'input_efficientnet': input_image_chest
})

print("Predicted Chest Size:", predicted_chest[0][0])

1/1 [==============================] - 6s 6s/step
Predicted Chest Size: 109.5414


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, concatenate, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data = {'path1': df['path1'],
        'path2': df['path2'],
        'height': df['height'],
        'waist': df['waist'],
        'chest': df['chest']}

df = pd.DataFrame(data)

# Function to load and preprocess images
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size, color_mode='rgb')
    img_array = image.img_to_array(img)
    img_array /= 255.0  # Rescale pixel values
    return img_array

# Apply the function to load and preprocess images
df['image1'] = df['path1'].apply(lambda x: load_and_preprocess_image(x))
df['image2'] = df['path2'].apply(lambda x: load_and_preprocess_image(x))

# Create image arrays for waist prediction
X1 = np.stack(df['image1'].to_numpy())
X2 = np.stack(df['image2'].to_numpy())
y_waist = df['waist'].to_numpy()

# Split the dataset into training and validation sets for waist size
X1_train, X1_val, X2_train, X2_val, y_waist_train, y_waist_val = train_test_split(
    X1, X2, y_waist, test_size=0.2, random_state=42
)

# Define the VGG16 model with two inputs for waist prediction
vgg16_model_waist = VGG16(weights='imagenet', include_top=False, input_shape=input_shape_vgg16)

# Freeze the convolutional layers
for layer in vgg16_model_waist.layers:
    layer.trainable = False

# Define the ResNet50 model with two inputs for waist prediction
resnet50_model_waist = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape_resnet50)

# Freeze the convolutional layers
for layer in resnet50_model_waist.layers:
    layer.trainable = False

# Define the DenseNet121 model with two inputs for waist prediction
densenet_model_waist = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape_densenet)

# Freeze the convolutional layers
for layer in densenet_model_waist.layers:
    layer.trainable = False

# Define the EfficientNetB0 model with two inputs for waist prediction
efficientnet_model_waist = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape_efficientnet)

# Freeze the convolutional layers
for layer in efficientnet_model_waist.layers:
    layer.trainable = False

# Combine the four models for waist prediction
vgg16_features_waist = vgg16_model_waist(input_vgg16)
resnet50_features_waist = resnet50_model_waist(input_resnet50)
densenet_features_waist = densenet_model_waist(input_densenet)
efficientnet_features_waist = efficientnet_model_waist(input_efficientnet)

# Concatenate the output features for waist prediction
merged_waist = concatenate([Flatten()(vgg16_features_waist), Flatten()(resnet50_features_waist),
                            Flatten()(densenet_features_waist), Flatten()(efficientnet_features_waist)])

# Dense layers for further processing for waist prediction
x_waist = Dense(128, activation='relu')(merged_waist)
x_waist = Dense(64, activation='relu')(x_waist)

# Separate output layer for waist size
output_waist = Dense(1, name='output_waist')(x_waist)

# Create the combined model with a single output for waist prediction
combined_waist_model = Model(
    inputs=[input_vgg16, input_resnet50, input_densenet, input_efficientnet],
    outputs=[output_waist]
)

# Compile the model for waist prediction
optimizer = Adam(learning_rate=0.001)
combined_waist_model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])

# Train the model for waist prediction
history_waist = combined_waist_model.fit(
    {'input_vgg16': X2_train, 'input_resnet50': X2_train, 'input_densenet': X2_train, 'input_efficientnet': X2_train},
    {'output_waist': y_waist_train},
    epochs=30,
    batch_size=32,
    validation_data=(
        {
            'input_vgg16': X2_val,
            'input_resnet50': X2_val,
            'input_densenet': X2_val,
            'input_efficientnet': X2_val
        },
        {'output_waist': y_waist_val}
    )
)


Epoch 1/30
13/13 [==============================] - 29s 960ms/step - loss: 1997.2162 - mean_absolute_error: 34.1921 - val_loss: 215.2103 - val_mean_absolute_error: 10.1505
Epoch 2/30
13/13 [==============================] - 6s 490ms/step - loss: 207.2317 - mean_absolute_error: 11.8505 - val_loss: 204.5343 - val_mean_absolute_error: 12.6308
Epoch 3/30
13/13 [==============================] - 6s 503ms/step - loss: 137.9655 - mean_absolute_error: 9.5691 - val_loss: 159.2752 - val_mean_absolute_error: 10.8950
Epoch 4/30
13/13 [==============================] - 6s 495ms/step - loss: 66.5125 - mean_absolute_error: 6.4883 - val_loss: 126.7848 - val_mean_absolute_error: 9.3007
Epoch 5/30
13/13 [==============================] - 6s 502ms/step - loss: 57.9650 - mean_absolute_error: 6.0431 - val_loss: 118.5789 - val_mean_absolute_error: 7.8136
Epoch 6/30
13/13 [==============================] - 6s 493ms/step - loss: 43.0037 - mean_absolute_error: 5.2219 - val_loss: 106.7645 - val_mean_absolute_er

In [ ]:

image_path_waist = "/content/desktop/testB/mask/6eedf087dddd033a288bf64a6e19d1b9.png"

# Load and preprocess the image for waist prediction
input_image_waist = load_and_preprocess_image(image_path_waist)

# Expand dimensions to match the input shape
input_image_waist = np.expand_dims(input_image_waist, axis=0)

# Make the prediction for waist size
predicted_waist = combined_waist_model.predict({
    'input_vgg16': input_image_waist,
    'input_resnet50': input_image_waist,
    'input_densenet': input_image_waist,
    'input_efficientnet': input_image_waist
})

# Print the predicted waist size
print("Predicted Waist Size:", predicted_waist[0][0])


1/1 [==============================] - 0s 57ms/step
Predicted Waist Size: 82.380135


In [ ]:
pip install bardapi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 51.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 81.0 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=3923c5713824a0770011bacb36e6b828ba14f2cfdc460387524dc36e4f31279c
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
from bardapi import Bard

In [ ]:
str1=input("What Brand you are buying- ")
str2=input("what are you buying ")

What Brand you are buying- Calvin Klein
what are you buying T-Shirt


In [ ]:
print(str1)

Calvin Klein


In [ ]:
token1='eAjMITXClX5sALd8zbQbyhcruzoBwt6ffPDy-28TLeqLAbkY9Ow1csL2JzpITv2cBG0EEA.'
bard=Bard(token1)
result1=bard.get_answer(f"print me the Men's size chart of {str1}")
result=bard.get_answer(f"if height of the particular person is {predicted_height[0][0]} , waist size is {predicted_waist[0][0]} and chest size is {predicted_chest[0][0]} and want to buy {str1} {str2}, so what will be size one should buy and brand size chart is {result1}")
print(result)

{'content': "Based on the size chart you provided, the recommended size for a person with a height of 168cm, waist size of 88cm, and chest size of 49cm is **L**.\n\nHowever, it's important to note that this is just a recommendation, and the best way to ensure a good fit is to try on the shirt in person. If you are unable to try on the shirt in person, you may want to consider ordering a size up or down, depending on how you prefer your clothes to fit.\n\nHere is a breakdown of the Calvin Klein T-Shirt size chart:\n\n| Size | Chest (cm) | Waist (cm) |\n|---|---|---|\n| XS | 88-91 | 68-71 |\n| S | 94-97 | 74-76 |\n| M | 99-102 | 79-81 |\n| L | 104-107 | 84-86 |\n| XL | 109-112 | 89-91 |\n| XXL | 114-117 | 94-96 |\n\nAs you can see, the L size is the closest fit for your chest size and waist size. However, if you prefer a looser fit, you may want to consider ordering an XL.\n\nHere are some additional tips for choosing the right size Calvin Klein T-Shirt:\n\n* **Measure your chest and wai